# beautifulsoup for scraping tsa information

In [223]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# --- Scraping tsa data ---
url = "https://www.eia.gov/dnav/pet/hist/leafhandler.ashx?f=m&n=pet&s=emm_epm0_pte_nus_dpg"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table))[0]

In [ ]:
# use xlrd openxl to convert xls files to csv for gas prices
pip install xlrd openpyxl
import pandas as pd
xls_file = "C:\Users\jstew\DE_projects\TSA-labor-day\Unemployment-rates.xlsx"
df = pd.read_excel(xls_file, sheet_name='Data 1', skiprows=1)

In [357]:
# viewing xls first 5 rows
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0
1,2016,4.8,4.9,5.0,5.1,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7
2,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.3,4.2,4.2,4.1
3,2018,4.0,4.1,4.0,4.0,3.8,4.0,3.8,3.8,3.7,3.8,3.8,3.9
4,2019,4.0,3.8,3.8,3.7,3.6,3.6,3.7,3.6,3.5,3.6,3.6,3.6


In [318]:
# convert df to csv file
df.to_csv('gas-prices-weekly.csv', index=False)

In [443]:
# use xlrd openxl to convert xls files to csv for unemployment rates
xls_file = "C:/Users/jstew/DE_projects/TSA-labor-day/Unemployment-rates.xlsx"
xls = pd.ExcelFile(xls_file)
print(xls.sheet_names)

df = pd.read_excel(xls_file, sheet_name='BLS Data Series', skiprows=11)
df.head()

['BLS Data Series']


C:\Users\jstew\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0
1,2016,4.8,4.9,5.0,5.1,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7
2,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.3,4.2,4.2,4.1
3,2018,4.0,4.1,4.0,4.0,3.8,4.0,3.8,3.8,3.7,3.8,3.8,3.9
4,2019,4.0,3.8,3.8,3.7,3.6,3.6,3.7,3.6,3.5,3.6,3.6,3.6


In [444]:
# save unemployment-rates as a csv
df.to_csv('unemployment-rates.csv', index=False)

In [445]:
df_unemployment = df

In [447]:
df_unemployment.dtypes

Year      int64
Jan     float64
Feb     float64
Mar     float64
Apr     float64
May     float64
Jun     float64
Jul     float64
Aug     float64
Sep     float64
Oct     float64
Nov     float64
Dec     float64
dtype: object

In [448]:
# Melt wide table (Jan, Feb, ...) into long format
df_unemp_long = df_unemployment.melt(
    id_vars=['Year'],
    value_vars=['Jan','Feb','Mar','Apr','May','Jun',
                'Jul','Aug','Sep','Oct','Nov','Dec'],
    var_name='Month',
    value_name='UnemploymentRate'
)

# Map month abbreviations to numbers
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
df_unemp_long['MonthNum'] = df_unemp_long['Month'].map(month_map)

# Create a datetime column (set to first of each month)
df_unemp_long['month_date'] = pd.to_datetime(
    df_unemp_long['Year'].astype(str) + '-' + df_unemp_long['MonthNum'].astype(str) + '-01'
)

# Final unemployment dataframe
unemployment = df_unemp_long[['month_date', 'UnemploymentRate']]

In [450]:
unemployment.dtypes

month_date          datetime64[ns]
UnemploymentRate           float64
dtype: object

In [394]:
# use xlrd openxl to convert xls files to csv for quarterly airfare pricing
xls_file = "C:/Users/jstew/DE_projects/TSA-labor-day/quarterly-airfare-rates.xlsx"
xls = pd.ExcelFile(xls_file)
print(xls.sheet_names)

df = pd.read_excel(xls_file, sheet_name='Fares', skiprows=0)
df.head()

['Fares']


,Year,Quarter,U.S. Average (Current $),U.S. Average (Inflation-Adjusted $)
0,1995,1,296.895553,627.476878
1,1995,2,296.462724,621.073174
2,1995,3,287.514840,599.701035
3,1995,4,287.776292,597.380616
4,1996,1,283.969692,584.154358


In [400]:
# round airfare pricing to the nearest cent
df = df.round(2)
print(df)

     Year  Quarter  U.S. Average (Current $)   \
0    1995        1                     296.90   
1    1995        2                     296.46   
2    1995        3                     287.51   
3    1995        4                     287.78   
4    1996        1                     283.97   
..    ...      ...                        ...   
116  2024        1                     387.95   
117  2024        2                     382.27   
118  2024        3                     365.65   
119  2024        4                     397.67   
120  2025        1                     397.16   

     U.S. Average (Inflation-Adjusted $)   
0                                  627.48  
1                                  621.07  
2                                  599.70  
3                                  597.38  
4                                  584.15  
..                                    ...  
116                                398.57  
117                                388.26  
118            

In [408]:
airfare_df = pd.read_csv('quarterly-airfare-rates.csv', skiprows=0)

In [410]:
print(airfare_df.columns)

Index(['Year', 'Quarter', 'U.S. Average (Current $) ',
       'U.S. Average (Inflation-Adjusted $) '],
      dtype='object')


In [411]:
# Rename columns to remove spaces and simplify
airfare_df = airfare_df.rename(columns={
    'Year': 'year',
    'Quarter': 'quarter',
    'U.S. Average (Current $) ': 'avg_current',
    'U.S. Average (Inflation-Adjusted $) ': 'avg_inflation'
})

In [412]:
# Now you can create a datetime from year and quarter
airfare_df['quarter_date'] = pd.to_datetime(
    airfare_df['year'].astype(str) + 'Q' + airfare_df['quarter'].astype(str)
)

C:\Users\jstew\AppData\Local\Temp\ipykernel_5832\2858479198.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [413]:
quarter_to_month = {'1': '01', '2': '04', '3': '07', '4': '10'}

In [427]:
# Create a proper datetime from year + quarter
airfare_df['quarter_date'] = pd.to_datetime(
    airfare_df['year'].astype(str) + '-' +
    airfare_df['quarter'].astype(str).map(quarter_to_month) + '-01'
)

KeyError: 'quarter'

In [428]:
print(airfare_df)

     year  avg_inflation quarter_date
0    1995         627.48   1995-01-01
1    1995         621.07   1995-04-01
2    1995         599.70   1995-07-01
3    1995         597.38   1995-10-01
4    1996         584.15   1996-01-01
..    ...            ...          ...
116  2024         398.57   2024-01-01
117  2024         388.26   2024-04-01
118  2024         370.27   2024-07-01
119  2024         401.78   2024-10-01
120  2025         397.16   2025-01-01

[121 rows x 3 columns]


In [431]:
print(airfare_df)

     year  avg_inflation quarter_date
0    1995         627.48   1995-01-01
1    1995         621.07   1995-04-01
2    1995         599.70   1995-07-01
3    1995         597.38   1995-10-01
4    1996         584.15   1996-01-01
..    ...            ...          ...
116  2024         398.57   2024-01-01
117  2024         388.26   2024-04-01
118  2024         370.27   2024-07-01
119  2024         401.78   2024-10-01
120  2025         397.16   2025-01-01

[121 rows x 3 columns]


In [432]:
# convert airfare df to csv file
df.to_csv('airfare-rates.csv', index=False)

In [434]:
airfare_df.dtypes

year                      int64
avg_inflation           float64
quarter_date     datetime64[ns]
dtype: object

In [436]:
airfare_df = airfare_df.sort_values('quarter_date')

In [381]:
#using glob to combine all tsa csv files to one large file for all years
import pandas as pd
import glob

files = glob.glob("tsa_data_*.csv")
print("Merging these files:", files)

tsa_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
tsa_df.to_csv("tsa_data_all_years.csv", index=False)

Merging these files: ['tsa_data_2019.csv', 'tsa_data_2020.csv', 'tsa_data_2021.csv', 'tsa_data_2022.csv', 'tsa_data_2023.csv', 'tsa_data_2024.csv', 'tsa_data_2025.csv']


In [382]:
# viewing first 5 rows of tsa new csv file
tsa_df.head()

,date,numbers
0,2019-01-01,2201765
1,2019-01-02,2424225
2,2019-01-03,2279384
3,2019-01-04,2230078
4,2019-01-05,2049460


In [390]:
df = pd.read_csv('tsa_data_all_years.csv')
df.head()

,date,numbers
0,2019-01-01,2201765
1,2019-01-02,2424225
2,2019-01-03,2279384
3,2019-01-04,2230078
4,2019-01-05,2049460


In [ ]:
# using to_datetime to change type to datetime for tsa csv
tsa['date'] = pd.to_datetime(tsa['date'])

# Rename columns
gas = pd.read_csv('gas-prices-weekly.csv', skiprows=1)

In [ ]:
# changing gas csv column names for better reading
gas.columns = ['week_start', 'price']

# using to_datetime to change type to datetime for gas csv
gas['week_start'] = pd.to_datetime(gas['week_start'])

In [435]:
# viewing first five rows of gas csv
gas.head()

,week_start,price
0,1993-04-05,1.068
1,1993-04-12,1.079
2,1993-04-19,1.079
3,1993-04-26,1.086
4,1993-05-03,1.086


In [ ]:
# saving update gas csv
gas.to_csv('gas-prices-weekly.csv', index=False)

In [ ]:
# viewing tsa and gas data types
tsa.dtypes
gas.dtypes

In [ ]:
# sorting gas and tsa by date chronologically
gas = gas.sort_values('week_start')
tsa = tsa.sort_values('date')

In [451]:
# merge tsa and gas csv to one csv
tsa_gas_merged = pd.merge_asof(
    tsa,                # daily TSA data
    gas,                # weekly gas data
    left_on='date',     # TSA daily date
    right_on='week_start'  # weekly gas week start
)

In [454]:
# Merge unemployment (monthly)
tsa_gas_unemp_merged = pd.merge_asof(
    tsa_gas_merged.sort_values('date'),
    unemployment.sort_values('month_date'),
    left_on='date',
    right_on='month_date'
)

In [455]:
# Merge airfare (quarterly)
final_merged = pd.merge_asof(
    tsa_gas_unemp_merged.sort_values('date'),
    airfare_df.sort_values('quarter_date'),
    left_on='date',
    right_on='quarter_date'
)

In [456]:
print(final_merged)

           date  numbers week_start  price month_date  UnemploymentRate  year  \
0    2019-01-01  2201765 2018-12-31  2.358 2019-01-01               4.0  2019   
1    2019-01-02  2424225 2018-12-31  2.358 2019-01-01               4.0  2019   
2    2019-01-03  2279384 2018-12-31  2.358 2019-01-01               4.0  2019   
3    2019-01-04  2230078 2018-12-31  2.358 2019-01-01               4.0  2019   
4    2019-01-05  2049460 2018-12-31  2.358 2019-01-01               4.0  2019   
...         ...      ...        ...    ...        ...               ...   ...   
2413 2025-08-10  2960349 2025-08-04  3.265 2025-08-01               NaN  2025   
2414 2025-08-11  2763671 2025-08-11  3.243 2025-08-01               NaN  2025   
2415 2025-08-12  2256082 2025-08-11  3.243 2025-08-01               NaN  2025   
2416 2025-08-13  2331215 2025-08-11  3.243 2025-08-01               NaN  2025   
2417 2025-08-14  2832747 2025-08-11  3.243 2025-08-01               NaN  2025   

      avg_inflation quarter

In [457]:
final_merged.to_csv("final_merged.csv", index=False)

In [379]:
# dropping week_start column
tsa_gas_merged = tsa_gas_merged.drop(columns=['week_start'])
tsa_gas_merged.head()

KeyError: "['week_start'] not found in axis"

In [250]:
# renamed columns from numbers to passengers
tsa_gas_merged = tsa_gas_merged.rename(columns={'numbers': 'passengers'})

In [ ]:
# save tsa gas merged csv
tsa_gas_merged.to_csv('tsa_gas_merged.csv', index=False)

In [252]:
# merge airfare and gas/tsa csvs to one csv
tsa_gas_merged = pd.merge_asof(
    tsa,                # daily TSA data
    gas,                # weekly gas data
    left_on='date',     # TSA daily date
    right_on='week_start'  # weekly gas week start
)

In [ ]:
# Merge TSA+gas with monthly unemployment
tsa_gas_unemp = pd.merge_asof(
    tsa_gas_merged.sort_values('date'),
    unemployment.sort_values('month_date'),   # make sure you created a datetime col for year+month
    left_on='date',
    right_on='month_date'
)

In [253]:
# prophet for forecasting
pip install prophet


     ---------------------------------------- 0.0/46.6 kB ? eta -:--:--
     -------- ------------------------------- 10.2/46.6 kB ? eta -:--:--
     ------------------------- ------------ 30.7/46.6 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 46.6/46.6 kB 386.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.2/13.3 MB 3.9 MB/s eta 0:00:04
   - -------------------------------------- 0.4/13.3 MB 3.9 MB/s eta 0:00:04
   - -------------------------------------- 0.6/13.3 MB 4.4 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/13.3 MB 5.5 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/13.3 MB 6.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.9/13.3 MB 6.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/13.3 MB 7.1 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/13.3 MB 7.2 MB/s eta 0:00:02
   -----

In [277]:
from prophet import Prophet

In [290]:
model = Prophet(yearly_seasonality=True, weekly_seasonality=True)

In [291]:
model.fit(tsa_df)

20:51:31 - cmdstanpy - INFO - Chain [1] start processing
20:51:32 - cmdstanpy - INFO - Chain [1] done processing


In [292]:
future = model.make_future_dataframe(periods=20)

In [293]:
forecast = model.predict(future)

In [297]:
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(21)

,ds,yhat,yhat_lower,yhat_upper
2417,2025-08-14,2.723999e+06,2.486343e+06,2.973343e+06
2418,2025-08-15,2.760592e+06,2.505638e+06,3.004229e+06
2419,2025-08-16,2.416302e+06,2.163216e+06,2.662827e+06
2420,2025-08-17,2.730065e+06,2.495935e+06,2.976075e+06
2421,2025-08-18,2.642867e+06,2.387578e+06,2.878395e+06
2422,2025-08-19,2.314934e+06,2.053618e+06,2.558438e+06
2423,2025-08-20,2.393428e+06,2.140778e+06,2.638738e+06
2424,2025-08-21,2.635371e+06,2.385566e+06,2.890792e+06
2425,2025-08-22,2.678491e+06,2.445300e+06,2.917516e+06
2426,2025-08-23,2.341193e+06,2.091880e+06,2.566111e+06


In [300]:
predicted_passengers = int(forecast[forecast['ds']=='2025-08-14']['yhat'].values[0])

In [301]:
print(predicted_passengers)

2723999
